## Few-shot examples for chat models

### Fixed Examples

In [1]:
from langchain_community.llms import Ollama
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]


# This is a prompt template used to format each individual example.

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

llama3_llm = Ollama(model="llama3")

chain = final_prompt | llama3_llm

chain.invoke({"input": "6+6"})

'My young apprentice, you seek the wisdom of numbers, do you not?\n\nAs a wondrous wizard of math, I shall conjure up the answers for thee!\n\nFor thy first query, "2+2", I reply... (waves staff) Ah, yes! The answer is indeed 4!\n\nAnd for thy second inquiry, "2+3", I respond... (murmurs incantation) Behold! The correct tally is 5!\n\nAnd finally, for thy third request, "6+6", I proclaim... (thunders forth) Lo and behold! The sum of the ages is 12!\n\nThus, the wisdom of numbers is revealed unto thee, my young friend! May it guide thee on thy journey through the realms of arithmetic!'

### Dynamic few-shot prompting

In [2]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from  langchain_openai import OpenAIEmbeddings


# Since we are using a vectorstore to select examples based on semantic similarity, we will want to first populate the store.
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vecrotize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vecrotize, embeddings, metadatas=examples)

In [3]:
# With a vectorstore created, you can create the example_selector. Here we will isntruct it to only fetch the top 2 examples.
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2+4', 'output': '6'}]

In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)

# Define the few-shot prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variable select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [
            ("human", "{input}"),
            ("ai", "{output}")
        ]
    )
)

few_shot_prompt.format(input="what is 3+3")

'Human: 2+4\nAI: 6\nHuman: What did the cow say to the moon?\nAI: nothing at all'

In [8]:
# Assemble the final prompt template

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very good at math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

final_prompt.format(input="what is 3+4")

'System: You are very good at math.\nHuman: 2+2\nAI: 4\nHuman: 2+4\nAI: 6\nHuman: what is 3+4'

In [9]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI()

chain = final_prompt | openai_llm

chain.invoke({"input": "what is 3+4"})

AIMessage(content='7', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 48, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6e9a41d4-703f-48bc-b008-472bc2dd6e1e-0')

In [10]:
chain.invoke({"input": "what is lion"})

AIMessage(content='A lion is a large carnivorous mammal belonging to the cat family.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 53, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7196d124-5ee4-42fa-9d44-3e701860ce91-0')